In [19]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import random
import os

# 1. 현재 경로 설정
os.chdir(os.path.dirname(os.path.abspath(__file__))) if '__file__' in locals() else None

# 파일명
file_name = '한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv'
file_path = './' + file_name

if os.path.exists(file_path):
    print(f"✅ 데이터 로딩 중... ({file_name})")
    
    # ⚡ 꿀팁: low_memory=False를 넣으면 경고 메시지가 사라지고 속도가 빨라집니다.
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

    # 데이터 전처리
    df[['위도', '경도']] = df['위도경도'].str.split(',', expand=True)
    df['위도'] = pd.to_numeric(df['위도'].str.strip(), errors='coerce')
    df['경도'] = pd.to_numeric(df['경도'].str.strip(), errors='coerce')
    
    # 필요한 데이터만 남기기 (메모리 절약)
    df = df[['충전소명', '주소', '위도', '경도', '시도']].dropna()

    # 지도 생성
    m = folium.Map(location=[36.5, 127.5], zoom_start=7)

    # 색상 설정
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue',
              'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    
    unique_sidos = df['시도'].unique()
    if len(unique_sidos) > len(colors):
        colors = colors * (len(unique_sidos) // len(colors) + 1)
    
    random.shuffle(colors)
    # 딕셔너리 접근 속도를 높이기 위해 미리 생성
    color_map = {sido: colors[i % len(colors)] for i, sido in enumerate(unique_sidos)}

    # 마커 클러스터 생성
    marker_cluster = MarkerCluster().add_to(m)

    print(f"📍 {len(df)}개의 마커를 생성 중입니다... (zip 방식 적용)")

    # 🔥 [핵심 수정] iterrows() 대신 zip() 사용 -> 속도 10배 향상 🔥
    # 판다스 내부 함수를 쓰지 않고 파이썬 리스트로 변환해서 돌리기 때문에 훨씬 빠릅니다.
    for lat, lon, name, addr, sido in zip(df['위도'], df['경도'], df['충전소명'], df['주소'], df['시도']):
        folium.Marker(
            location=[lat, lon],
            popup=f"{name}<br>{addr}",
            icon=folium.Icon(color=color_map.get(sido, 'gray'))
        ).add_to(marker_cluster)

    # 저장
    output_file = 'ev_charging_stations_map_fast.html'
    m.save(output_file)
    print(f"🎉 지도 저장 완료! 파일명: {output_file}")
    
else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_name}")

✅ 데이터 로딩 중... (한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv)
📍 347847개의 마커를 생성 중입니다... (zip 방식 적용)
🎉 지도 저장 완료! 파일명: ev_charging_stations_map_fast.html


In [ ]:
from google.colab import files

files.download('ev_charging_stations_map.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df

,충전소명,주소,위도,경도,시도
0,가로공원로 지하공영주차장,서울특별시 강서구 가로공원로 189,37.537263,126.838379,서울특별시
1,곰달래문화복지센터 공영주차장,서울특별시 강서구 강서로5길 50,37.528956,126.849089,서울특별시
2,볏골공원 지하공영주차장,서울특별시 강서구 까치산로4길 22,37.541975,126.844450,서울특별시
3,화곡6-1 공영주차장,서울특별시 강서구 우장산로 114,37.553046,126.848817,서울특별시
4,신림동 제1공영주차장,서울특별시 관악구 관천로 98,37.487405,126.923507,서울특별시
...,...,...,...,...,...
361144,부산 시민공원삼정그린코아,부산광역시 부산진구 동평로 176,35.165837,129.049660,부산광역시
361145,부산 시민공원삼정그린코아,부산광역시 부산진구 동평로 176,35.165837,129.049660,부산광역시
361146,부산 시민공원삼정그린코아,부산광역시 부산진구 동평로 176,35.165837,129.049660,부산광역시
361147,부산 시민공원삼정그린코아,부산광역시 부산진구 동평로 176,35.165837,129.049660,부산광역시


# Task
Analyze and visualize the density of EV charging stations in specific areas using the data from "/content/ev_charging_stations_map.html".

## 데이터 준비

### Subtask:
밀집도 분석에 필요한 위도, 경도 데이터를 준비합니다.


In [21]:
# Step 1: Select '위도' and '경도' columns
coords_df = df[['위도', '경도']]

# Step 2: Display the first few rows of the new DataFrame
display(coords_df.head())

,위도,경도
0,37.537263,126.838379
1,37.528956,126.849089
2,37.541975,126.844450
3,37.553046,126.848817
4,37.487405,126.923507


## 밀집도 계산

### Subtask:
특정 지역별 또는 격자별 충전소 밀집도를 계산합니다.


In [22]:
# Step 1: Group the data by '시도' and count the number of stations in each '시도'
density_by_sido = df['시도'].value_counts().reset_index()

# Step 2: Rename the columns for clarity
density_by_sido.columns = ['시도', '충전소 수']

# Step 3: Display the result
display(density_by_sido)

,시도,충전소 수
0,경기도,96357
1,서울특별시,56351
2,부산광역시,21481
3,인천광역시,19197
4,경상남도,19017
5,대구광역시,17319
6,경상북도,15862
7,충청남도,15078
8,강원특별자치도,12431
9,전북특별자치도,12123


## 데이터 시각화

### Subtask:
계산된 밀집도 데이터를 지도 위에 히트맵(heatmap) 또는 클러스터 맵(cluster map) 형태로 시각화합니다.


In [24]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import os

# 1. 경로 및 파일 설정
os.chdir(os.path.dirname(os.path.abspath(__file__))) if '__file__' in locals() else None
file_name = '한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv'
file_path = './' + file_name

if os.path.exists(file_path):
    print(f"✅ 데이터 로딩 및 전처리 중... ({file_name})")
    
    # 데이터 읽기
    df = pd.read_csv(file_path, encoding='cp949', low_memory=False)

    # 전처리: 위도, 경도 분리 및 숫자 변환
    df[['위도', '경도']] = df['위도경도'].str.split(',', expand=True)
    df['위도'] = pd.to_numeric(df['위도'].str.strip(), errors='coerce')
    df['경도'] = pd.to_numeric(df['경도'].str.strip(), errors='coerce')
    
    # 📌 Step 3: 히트맵 데이터 준비 (위도, 경도만 추출하여 리스트로 변환)
    # NaN 값 제거 후 리스트 변환 (히트맵은 결측치가 있으면 에러 발생 가능)
    coords_df = df[['위도', '경도']].dropna()
    heatmap_data = coords_df.values.tolist()

    print(f"🔥 히트맵 데이터 준비 완료: {len(heatmap_data)}개의 포인트")

    # 📌 Step 2: 지도 생성
    m_heatmap = folium.Map(location=[36.5, 127.5], zoom_start=7)

    # 📌 Step 4: 히트맵 레이어 추가
    # radius: 점의 크기, blur: 흐림 정도 (조절하여 시각화 효과 변경 가능)
    HeatMap(heatmap_data, radius=15, blur=20, max_zoom=10).add_to(m_heatmap)

    # 📌 Step 5: 저장
    output_file_heatmap = 'ev_charging_heatmap.html'
    m_heatmap.save(output_file_heatmap)
    
    print(f"🎉 히트맵 저장 완료! 파일명: {output_file_heatmap}")

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_name}")

✅ 데이터 로딩 및 전처리 중... (한국환경공단_전기차 충전소 위치 및 운영정보(충전소 ID 포함)_20230531.csv)
🔥 히트맵 데이터 준비 완료: 347847개의 포인트
🎉 히트맵 저장 완료! 파일명: ev_charging_heatmap.html


## 결과 해석

### Subtask:
시각화된 지도를 통해 충전소 밀집도가 높은 지역을 파악하고 결과를 해석합니다.
